In [ ]:
# Mounting the Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/qna_package_nlp_final/

/content/drive/MyDrive/qna_package_nlp_final


# Downloading squad_v2 Data

Method 1

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset,load_metric
raw_datasets = load_dataset("squad_v2")
metric = load_metric("rouge")

  0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-56-d1b9895e5926>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
# # qna model needs data in this json format

# {
#   "data": [
#     {
#       "title": "some text",
#       "context": "some text",
#       "question": "some question",
#       "answers": {
#         "text": [
#           "some text"
#         ],
#         "answer_start": [
#           45
#         ]
#       },
#       "id": "0"
#     },

### Saving train data in json format

In [ ]:
## inefficient
# from tqdm import tqdm
# import pandas as pd
# import json
# train_df = pd.DataFrame()
# for i in tqdm(raw_datasets['train']):
#   try: 
#     temp_df = pd.DataFrame({'id':[i['id']],'title':[i['title']],'question':[i['question']], 'context':[i['context']], 'answers':[{'text':[i['answers']['text'][0]]}]})
#   except:
#     temp_df = pd.DataFrame({'id':[i['id']],'title':[i['title']],'question':[i['question']], 'context':[i['context']], 'answers':[{'text':[""]}]})
#   train_df = pd.concat([train_df,temp_df])
# train_df = train_df.reset_index().drop(columns=['index'])

# train_df = train_df.to_json(orient="records")
# train_df = json.loads(train_df)
# train_df = {'data':train_df}

# with open('usecase_data/train_data_squadv2.json', 'w', encoding='utf-8') as f:
#     json.dump(train_df, f, ensure_ascii=False, indent=4)

In [ ]:
import pandas as pd
import json


In [ ]:
%%time
temp=[]
for i in range(len(raw_datasets['train'])):
  temp.append(raw_datasets['train'][i])
train_df=pd.DataFrame(temp)
train_df = train_df.reset_index().drop(columns=['index'])
train_df = train_df.to_json(orient="records")
train_df = json.loads(train_df)
train_df = {'data':train_df}

with open('usecase_data/train_data_squadv2.json', 'w', encoding='utf-8') as f:
    json.dump(train_df, f, ensure_ascii=False, indent=4)

CPU times: user 15.8 s, sys: 237 ms, total: 16 s
Wall time: 16 s


### Saving validation data in json format

In [ ]:
temp=[]
for i in range(len(raw_datasets['validation'])):
  temp.append(raw_datasets['validation'][i])
val_df=pd.DataFrame(temp)
val_df = val_df.reset_index().drop(columns=['index'])

In [ ]:
val_df.head(2)

,id,title,context,question,answers
0,56ddde6b9a695914005b9628,Normans,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,"{'text': ['France', 'France', 'France', 'Franc..."
1,56ddde6b9a695914005b9629,Normans,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"{'text': ['10th and 11th centuries', 'in the 1..."


taking frst 500 rows as test data and remaining data to val data

In [ ]:
test_df = val_df[:500].to_json(orient="records")
test_df = json.loads(test_df)
test_df = {'data':test_df}
with open('usecase_data/test_data_squadv2.json', 'w', encoding='utf-8') as f:
    json.dump(test_df, f, ensure_ascii=False, indent=4)

In [ ]:
val_df_final = val_df[501:].to_json(orient="records")
val_df_final = json.loads(val_df_final)
val_df_final = {'data':val_df_final}

with open('usecase_data/validation_data_squadv2.json', 'w', encoding='utf-8') as f:
    json.dump(val_df_final, f, ensure_ascii=False, indent=4)

Method 2:

https://rajpurkar.github.io/SQuAD-explorer/

::we can download train and validation json data from the above link and change the json format for qna model .eg:data shd be there in json format. csv it throws an error

In [ ]:
from pandas import json_normalize
import json
data=open('usecase_data/train-v2.0.json')
raw_df = json.load(data)                                                      
raw_df=json_normalize(raw_df['data'])
raw_df.head(2)

,title,paragraphs
0,Beyoncé,[{'qas': [{'question': 'When did Beyonce start...
1,Frédéric_Chopin,"[{'qas': [{'question': ""What was Frédéric's na..."


# Question - Answering System

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-y3106w4v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-y3106w4v
  Resolved https://github.com/huggingface/transformers to commit 996f127a904e24cfb1722db00bc1e84520b7d4d3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468

In [ ]:
from question_answer import QnA



## Initialize the class

Created a package QnA for fine tuning and prediction

In [ ]:
qna=QnA()

## Fine tuning

Use Colab with GPU accelerators to accelerate the training process

This cell trains the given model on the given dataset and performs the evaluation.

* The pre-trained model from huggingface which needs to be fine tuned on the dataset.

* The user should provide the training and validation file path(optional) in json/csv format.

* The user should also provide the output path where all the model results will be saved.

* If the model name is not specified in the function,default model**(deepset/tinyroberta-squad2)** is taken into consideration

We can pass any kwargs inside train function. **(i.e doc_stride,learning_rate,..)**

For more information the user can refer to the following italicized text 

https://github.com/huggingface/transformers/tree/main/examples/pytorch/question-answering

In [ ]:
finetuned_model=qna.train(train_data_path='usecase_data/train_data_500_squadv2.json',output_path='usecase',model_name='deepset/roberta-base-squad2',valn_path='usecase_data/validation_data_500_squadv2.json')

Executed
Model Name: deepset/roberta-base-squad2
validation
training started


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Trainer output:  b"05/10/2023 05:13:15 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False\n05/10/2023 05:13:15 - INFO - __main__ - Training/evaluation parameters TrainingArguments(\n_n_gpu=1,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=True,\ndo_predict=False,\ndo_train=True,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},\nfsdp_min_num_params=0,\nfsdp_transformer_laye

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Evaluation output:  b"05/10/2023 05:18:15 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False\n05/10/2023 05:18:15 - INFO - __main__ - Training/evaluation parameters TrainingArguments(\n_n_gpu=1,\nadafactor=False,\nadam_beta1=0.9,\nadam_beta2=0.999,\nadam_epsilon=1e-08,\nauto_find_batch_size=False,\nbf16=False,\nbf16_full_eval=False,\ndata_seed=None,\ndataloader_drop_last=False,\ndataloader_num_workers=0,\ndataloader_pin_memory=True,\nddp_backend=None,\nddp_bucket_cap_mb=None,\nddp_find_unused_parameters=None,\nddp_timeout=1800,\ndebug=[],\ndeepspeed=None,\ndisable_tqdm=False,\ndo_eval=False,\ndo_predict=True,\ndo_train=False,\neval_accumulation_steps=None,\neval_delay=0,\neval_steps=None,\nevaluation_strategy=no,\nfp16=False,\nfp16_backend=auto,\nfp16_full_eval=False,\nfp16_opt_level=O1,\nfsdp=[],\nfsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},\nfsdp_min_num_params=0,\nfsdp_transformer_

In [ ]:
finetuned_model=qna.train(train_data_path='usecase_data/train_data_500_squadv2.json',output_path='usecase',model_name='deepset/roberta-base-squad2',valn_path='usecase_data/validation_data_500_squadv2.json',
                          max_answer_length=20,max_seq_length=296,per_device_train_batch_size=2,num_train_epochs=3,learning_rate=5e-5,n_best_size=15)

##Prediction using single context

We can use any pre trained/fine tuned model from Huggingface to infer on the test input. 

The user should provide:

* query/question
* context

* If the model name is not specified,default model**(deepset/tinyroberta-squad2)** is taken into consideration

We can pass any kwargs inside predict function. **(i.e doc_stride,max_answer_length,..)**

In [ ]:
context = "The US has passed the peak on new coronavirus cases, " \
          "President Donald Trump said and predicted that some states would reopen this month. " \
          "The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world."

question = "What was President Donald Trump's prediction?"

In [ ]:
# qna.predict(question=question,context=context,doc_stride=128,max_answer_length=20,learning_rate=3e-5,n_best_size=20)
qna.predict(question=question,context=context,model_name='deepset/roberta-base-squad2')

Executed
Model Name: deepset/roberta-base-squad2


[{'question': "What was President Donald Trump's prediction?",
  'context': 'The US has passed the peak on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month. The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.',
  'predicted_answer': 'some states would reopen this month',
  'model_name': 'deepset/roberta-base-squad2',
  'score': 0.5812742710113525,
  'start': 100,
  'end': 135}]

##Prediction using test data

We can pass any kwargs inside predict function. **(i.e doc_stride,learning_rate,..)**

In [ ]:
qna.predict(test_path="usecase_data/test_data_squadv2.json",output_path="usecase/test_prediction",model_name='deepset/roberta-base-squad2')

Executed
Model Name: deepset/roberta-base-squad2
error message Command '['python', 'run_qa.py', '--model_name_or_path', 'deepset/roberta-base-squad2', '--test_file', 'usecase_data/test_data_squadv2.json', '--do_predict', '--max_answer_length=50', '--per_device_train_batch_size=8', '--learning_rate=3e-05', '--num_train_epochs=2', '--max_seq_length=384', '--doc_stride=128', '--n_best_size=20', '--output_dir', 'usecase/test_prediction', '--overwrite_output_dir']' returned non-zero exit status 1.
2023-05-10 13:32:11.692754: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100%|██████████| 1/1 [00:00<00:00, 446.54it/s]
[INFO|configuration_utils.py:669] 2023-05-10 13:32:17,582 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--deepset--roberta-base-squad2/snapshots/e09df911dd96d8b052d2665dfbb309e9398a9d70/config.json
[INFO|configuration_utils.py:725] 2023-05-10 13:32:17,596 >> Model config RobertaConfig 

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
qna.predict(test_path="usecase_data/test_data_squadv2.json",output_path="usecase/test_prediction",model_name='deepset/roberta-base-squad2',
            max_answer_length=20,max_seq_length=296,per_device_train_batch_size=2,num_train_epochs=3,learning_rate=5e-5,n_best_size=15)